# BIO Distant Annotation file from entity list

THis notebook produces the BIO Annotation file using string matching, based on a given entity list or gazette

# Execution parameters

### a) Entity list

In [ ]:
# SAP PRODUCTS LIST file:
PRODUCTS_FOLDER = "sap-products/"
sap_products_file = "prod_full_sap-L1_v2.csv"

### b) Text corpus: in this case from SAP Corp-Wiki dataset:

In [ ]:
FOLDER = "corp_wiki_raw_texts/"
max_len = 256
file_from = 151
file_to = 161
file_number = np.arange(file_from, file_to)

# Install Dependencies

In [ ]:
%matplotlib inline
import pandas as pd                #Pandas - Data processing, data frames
pd.options.display.max_rows = 1000 #handy for showing truncated results
import matplotlib.pyplot as plt    #matplotlib - graficas
import numpy as np                 #Numpy - arrays y manejo de datos
# import seaborn as sns              #Seaborn - Data visualization
import re

In [ ]:
!pip install spacy 
!pip install -U spaCy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy import load
nlp = spacy.load('en_core_web_sm')

In [ ]:
from re import search


In [ ]:
# from spacy.gold import biluo_tags_from_offsets
from spacy.training import offsets_to_biluo_tags

# LOAD SAP-PRODUCTS (execute once) 

In [ ]:
# LOAD-ONCE:
# -------------------------------------------------------------------------
# 1) LOAD SAP-PRODUCT_ENTITIES:
# -------------------------------------------------------------------------
# PRODUCTS_FOLDER = "sap-products/"
# sap_products_file = "prod_full_sap-L1_v2.csv"

sap_ent_prod =[]
fileObject = open(PRODUCTS_FOLDER+sap_products_file, "r")
sap_ent_prod = fileObject.read()
sap_ent_prod = sap_ent_prod.split('\n')
# sap_ent_prod

# Convert to lower case:
sap_ent_prod_lwr = []
for p in sap_ent_prod:
    p = p.replace('\ufeff','')
    sap_ent_prod_lwr.append(p.lower())

sap_ent_prod_lwr = sorted(set(sap_ent_prod_lwr))
    
print(len(sap_ent_prod_lwr))    
sap_ent_prod_lwr[:5]

In [ ]:
# sap_ent_prod_lwr

# LOAD TEXT (for unit testing) 

In [ ]:
# FILE_FOLDER =  "corp_wiki_raw_texts/"
# !ls = "{FILE_FOLDER}"

In [ ]:
# FOLDER = "corp_wiki_raw_texts/"
# file_path_from = FOLDER+"corp-wiki-20"
# print('loading text from: ', file_path_from)
# fileObject = open(file_path_from, "r")
# text_string = fileObject.read()
# text_string = text_string.lower()
# text_string

# CLEANING FUNCTIONS (v.2)

In [ ]:
# -------------------------------------------------------------------------
# create function to remove numbers in brackets from text_string
# -------------------------------------------------------------------------
def generate_numbers_in_brackets_list(n=200):
    numbers_list = np.arange(1,n)
    brackets_numbers_list = []
    for i in numbers_list:
        string = '[' + str(i) + ']'
        brackets_numbers_list.append(string)
    return brackets_numbers_list

# -------------------------------------------------------------------------
# create function to replace a string with another string
# -------------------------------------------------------------------------
def replace_string_from_texts_list(string, strings_list, replace_by):
    for s in strings_list:
        new_string = string.replace(s, replace_by)
    return new_string

# -------------------------------------------------------------------------
# SPLIT DATA_STRING INTO SENTENCES
# -------------------------------------------------------------------------
def split_text_string_into_sentences_v1(text_string, max_len):
    text_sentences = []
    text_sentences_tmp = text_string.split('.')
    strg_carry_forward = '' 
    
    for s_ in text_sentences_tmp:
        s = strg_carry_forward + ' ' + s_
        nr_wrds = s.split()
        if len(nr_wrds) > 5:
            if len(s) <= max_len:
                s_nlts = s.strip()
                s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
                s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
                s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
                s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
                text_sentences.append(s_nlts)
                strg_carry_forward = ''
        
            else:  # s > max_len
                #truncated 1
                trunc1 = smart_truncate(s, length=max_len, suffix='')

                s_nlts = trunc1.strip()
                s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
                s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
                s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
                s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
                text_sentences.append(s_nlts)
                
                # truncated 2
                index_trunc1 = len(trunc1)
                trunc2 = s[index_trunc1:]
                nr_wrds = trunc2.split()
                if len(nr_wrds) > 5:
                    s_nlts = trunc2.strip()
                    s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
                    s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
                    s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
                    s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
                    text_sentences.append(s_nlts)

                    strg_carry_forward = ''
                else:
                    strg_carry_forward = ''
        else:
            strg_carry_forward = s
            
    return text_sentences

# -------------------------------------------------------------------------
# TRUNCATE A STRING WITHOUT BREAKING A WORD IN THE MODDLE
# -------------------------------------------------------------------------
def smart_truncate(content, length=100, suffix=''):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix
    
# -------------------------------------------------------------------------
# RESET SENTENCE NUMBERING OF ANNOTATED SENTENCES DATAFRAME
# -------------------------------------------------------------------------
def reset_annotated_file_sentence_numbering(df_data, n):
    df_data = df_data.reset_index(drop=True)
    previous_sentence = ''
    counter = 1
    sentence_counter = 'Sentence: '+str(n)+'_'+str(counter)

    for i in df_data.index: 
        if i == 0:
            previous_sentence = df_data['Sentence'][i]
            df_data['Sentence'][i] = 'Sentence: '+str(n)+'_1'

        else:
            if df_data['Sentence'][i] == previous_sentence:
                previous_sentence = df_data['Sentence'][i]
                df_data['Sentence'][i] = sentence_counter
            else:
                counter = counter + 1
                sentence_counter = 'Sentence: '+str(n)+'_'+str(counter)
                previous_sentence = df_data['Sentence'][i]
                df_data['Sentence'][i] = sentence_counter

    return df_data

# CLEANING FUNCTIONS v3.0

In [ ]:
def smart_truncate(content, length=100, suffix=''):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

In [ ]:
def truncate(s, max_len):
    trunc1 = smart_truncate(s, length=max_len, suffix='')
    s_nlts = trunc1.strip()
    s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
    s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
    s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
    s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
#     text_sentences.append(s_nlts)
     # truncated 2
    index_trunc1 = len(trunc1)
    trunc2 = s[index_trunc1:]
    return s_nlts, trunc2

In [ ]:
# # UNIT TESTING:
# s = 'the company says that the combination of sap cloud platform with sap business technology platform is all about connecting business processes and experiences so asug members can make confident decisions with integrity'
# print(len(s))

In [ ]:
# ## UNIT TESTING: TRUNCATE function
# max_len = 300

# tr1, tr2 = truncate(s, max_len)
# print('tr1: ', tr1)
# print(len(tr1))
# print('tr2: ', tr2)
# print(len(tr2))

In [ ]:
def eliminate_spaces(s):
    s_nlts = s.strip()
    s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
    s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
    s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
    s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
    return s_nlts

In [ ]:
def split_text_string_into_sentences_v2(text_string, max_len):
    text_sentences = []
    text_sentences_tmp = text_string.split('.')
    strg_carry_forward = '' 
    
    for s_ in text_sentences_tmp:
        s = strg_carry_forward + ' ' + s_
#         print(s)
        
        nr_wrds = s.split()
        
        if len(nr_wrds) > 5:
            if len(s) <= max_len: 
#                 print(len(s), ' <= ' , max_len)
                s = eliminate_spaces(s)
                text_sentences.append(s)
                strg_carry_forward = ''
            else:
#                 print(len(s), ' > ' , max_len)
            
                counter = 1
                while len(s) > max_len:
#                     print(counter)
                    tr1, tr2 = truncate(s, max_len)
                    tr1 = eliminate_spaces(tr1)
                    text_sentences.append(tr1)

                    s = tr2
#                     print('tr1: ', tr1)
#                     print('tr2: ', tr2)
#                     print('s: ', s)
                    
                if len(s) > 0 and len(s) < max_len:
#                     print('len(s) > 0 and len(s) < max_len:')
                    nr_wrds = s.split()
                    if len(nr_wrds) > 5:
                        s = eliminate_spaces(s)
                        text_sentences.append(s)
                        strg_carry_forward = ''
                    else:
                        strg_carry_forward = s
                counter = counter + 1
        else:
#             print('s has less than 5 words')
            strg_carry_forward = s
    
    if len(strg_carry_forward) > 0:
        strg_carry_forward = eliminate_spaces(strg_carry_forward)
        text_sentences.append(strg_carry_forward)
        
    
    return text_sentences
            

In [ ]:
# # UNIT TESTING: split_text_string_into_sentences_v3
# s = 'the company says that the combination of sap cloud platform with sap business technology platform is all about connecting business processes and experiences so asug members can make confident decisions with integrity. as sap gold partner delaware consulting and sap work together to implement s/4hana cloud for rehau which is the first s/4hana live customer in china.'
# # s = 'the company says that'
# print(len(s))

In [ ]:
# # UNIT TESTING: split_text_string_into_sentences_v3
# max_len = 128

# split_text_string_into_sentences_v2(s, max_len)

## New function to clean text and break into sentences

In [ ]:
# -------------------------------------------------------------------------
# CLEAN TEXT AND BREAK INTO SENTENCES
# -------------------------------------------------------------------------
def clean_text_string_and_break_into_sentences_v1(text_string, max_len):
    s = text_string.lower()

    s = re.sub("@\S+", " ", s)    #remove twitter account
    s = re.sub("https*\S+", " ", s)  #remove url
    s = s.replace("\t", " ")  # remove tabs
    s = s.replace("\n",". ")  # remove line-breaks

    # DELETE BEGGINING_OF_ARTICLE NUMBER STRING by DOT SPACE
    # s = "1965636085\ This must not b3 delet3d, but the number at the end yes 134411 the date of release was 23 March 2019\n2268135735\tthe following are the test cases/browser/platform combinations that were run."
    s = s.replace("\ ", " ")
    regex = re.compile(r'\d{10}')
    numbers_to_delete = regex.findall(s)
    numbers_to_delete
    for i in numbers_to_delete:
        s = s.replace(str(i), ". ")

    # DELETE VERSION NUMBERS
    # s = "sap hana 2.0 is the mew version. sap s/4hana 15.11 is the latest s4 version."                  
    regex = re.compile(r'\b\d+\.\d+\b')
    version_numbers = regex.findall(s)
    #version_numbers

    for i in version_numbers:
        s = s.replace(str(i), " ")

    # REMOVE / EXCEPT FROM IN S/4HANA
    # s = "sap hana is the in-memory database. sap s/4hana is the latest sap erp version. the test cases/browser/platform combinations that will be run"
    s = s.replace("s/4hana", "s4-hana")
    s = s.replace("s/4", "s-4")
    s = s.replace("/", " ")
    s = s.replace("s4-hana", "s4/hana")
    s = s.replace("s-4", "s/4")

    # OTHER CHARACTER REPLACEMENTS
    s = s.replace("...",". ")  # remove line-breaks
    s = s.replace(". ..",". ") # remove line-breaks
    s = s.replace(":"," ")     # remove line-breaks
    s = re.sub("#\S+", " ", s) # Remove hashtags
    s = re.sub("\'\w+", '', s) # Remove ticks and the next character

    brackets_numbers_list = generate_numbers_in_brackets_list(n=200)
    s = replace_string_from_texts_list(s, brackets_numbers_list,'')

    s = re.sub('[^A-Za-z0-9.]+', ' ', s)  # Remove all punctuations, except from .
    s = re.sub(r'\w*\d+\w*', '', s)       # Remove digits

    #replace strings
    s = s.replace("i.e", " ")  # remove i.e
    s = s.replace("run.", " ")  # remove run.
    s = s.replace(".zip", " ")  # remove .zip
    s = s.replace("server.xml", " ")  # remove server.xml
    s = s.replace(".yml", " ")  # remove .yml
    s = s.replace(".pai", " ")  # remove .pai
    s = s.replace(".xml", " ")  # remove .xml
    s = s.replace(".run", " ")  # remove .rum
    s = s.replace(".com", " ")  # remove .com

    s = re.sub('\s{2,}', " ", s)  # Replace the over spaces

    text_sentences = split_text_string_into_sentences_v1(s, max_len)
    
    return text_sentences

In [ ]:
# -------------------------------------------------------------------------
# CLEAN TEXT AND BREAK INTO SENTENCES v2.0
# -------------------------------------------------------------------------
def clean_text_string_and_break_into_sentences_v2(text_string, max_len):
    s = text_string.lower()

    s = re.sub("@\S+", " ", s)    #remove twitter account
    s = re.sub("https*\S+", " ", s)  #remove url
    s = s.replace("\t", " ")  # remove tabs
    s = s.replace("\n",". ")  # remove line-breaks

    # DELETE BEGGINING_OF_ARTICLE NUMBER STRING by DOT SPACE
    # s = "1965636085\ This must not b3 delet3d, but the number at the end yes 134411 the date of release was 23 March 2019\n2268135735\tthe following are the test cases/browser/platform combinations that were run."
    s = s.replace("\ ", " ")
    regex = re.compile(r'\d{10}')
    numbers_to_delete = regex.findall(s)
    numbers_to_delete
    for i in numbers_to_delete:
        s = s.replace(str(i), ". ")

    # DELETE VERSION NUMBERS
    # s = "sap hana 2.0 is the mew version. sap s/4hana 15.11 is the latest s4 version."                  
    regex = re.compile(r'\b\d+\.\d+\b')
    version_numbers = regex.findall(s)
    #version_numbers

    for i in version_numbers:
        s = s.replace(str(i), " ")

    # REMOVE / EXCEPT FROM IN S/4HANA
    # s = "sap hana is the in-memory database. sap s/4hana is the latest sap erp version. the test cases/browser/platform combinations that will be run"
    s = s.replace("s/4hana", "s4-hana")
    s = s.replace("s/4", "s-4")
    s = s.replace("/", " ")
    s = s.replace("s4-hana", "s4/hana")
    s = s.replace("s-4", "s/4")

    # OTHER CHARACTER REPLACEMENTS
    s = s.replace("...",". ")  # remove line-breaks
    s = s.replace(". ..",". ") # remove line-breaks
    s = s.replace(":"," ")     # remove line-breaks
    s = re.sub("#\S+", " ", s) # Remove hashtags
    s = re.sub("\'\w+", '', s) # Remove ticks and the next character

    brackets_numbers_list = generate_numbers_in_brackets_list(n=200)
    s = replace_string_from_texts_list(s, brackets_numbers_list,'')

    s = re.sub('[^A-Za-z0-9.]+', ' ', s)  # Remove all punctuations, except from .
    s = re.sub(r'\w*\d+\w*', '', s)       # Remove digits

    #replace strings
    s = s.replace("i.e", " ")  # remove i.e
    s = s.replace("run.", " ")  # remove run.
    s = s.replace(".zip", " ")  # remove .zip
    s = s.replace("server.xml", " ")  # remove server.xml
    s = s.replace(".yml", " ")  # remove .yml
    s = s.replace(".pai", " ")  # remove .pai
    s = s.replace(".xml", " ")  # remove .xml
    s = s.replace(".run", " ")  # remove .rum
    s = s.replace(".com", " ")  # remove .com

    s = re.sub('\s{2,}', " ", s)  # Replace the over spaces

    text_sentences = split_text_string_into_sentences_v2(s, max_len)
    
    return text_sentences

In [ ]:
# -------------------------------------------------------------------------
# FIND SUB-STRING IN STRING (ready for multiple cases)
# -------------------------------------------------------------------------
import re

def find_entity_matches(sentence, ent):
    entity_match_indexes = [m.start() for m in re.finditer(r'{}'.format(re.escape(ent)), sentence)]
    return entity_match_indexes

# # UNIT TESTING:
# sentence = 'branding cortez will be branded as follows cr for install and vs integration package sap crystal reports version for visual studio cr runtime install package sap crystal reports runtime engine for'
# text_sentences = ['branding cortez will be branded as follows cr for install and vs integration package sap crystal reports version for visual studio cr runtime install package sap crystal reports runtime engine for']
# ent = 'sap crystal reports' 
# entity_match_indexes = find_entity_matches(sentence, ent)
# entity_match_indexes

In [ ]:
# UNIT TEST:

In [ ]:
# txt = '1920507669	Digital River order flow for "Crystal Products" via bobj platform Example Products Lumira Crystal products without maitenance OpenSAP Digital River order flow for "Subscription Products" via sapsubs platform Note: For checkouts where user is not charged like for trials, TSC for privileged user and free edition checkout, payment section is hidden (hence no DR iframe in Checkout V2). Example Products DCRM HCP TSC Solution Manager products Digital River order flow for termination Digital River order flow for change quantity (Upsell/Downsell) Note: For upsell orders, quantity change is applicable at the same time. For down-sell orders, quantity change is applicable in the next renewal cycle. 1824109303	This page contains high level concept of Fulfillment Hub defining main goals and boundaries of this system. Main goals and responsibilities Main pain points with current architecture: Each back-end system has heavy tightly coupled logic currently to support fulfillment and many cases actions need to be done across few systems. no easy end to end monitoring of order fulfillment, problems are usually discovered when user complains on-boarding of new products usually means development and release cycle in all involved systems what lead to long time to market cycle Goals: Be able to deploy changes in each system independently, deliver new fulfillment processes with more agility than currently, changes of the process should not lead to development and deployment in respective systems User Experience - faster fulfillment to the customer without waiting on CRM/ERP order processing, user should get his access to ordered products in minutes End to End Monitoring - Ability to track down fulfillment problems to see where in the process the fulfillment for a customer order is stuck, proactively monitor and escalate issues easier product on-boarding process Transparency of fulfillment process status to any system consuming this information Responsibilities: execute and monitor order fulfillment and termination process orchestrate actions of all legacy systems together with triggering provisioning/transaction emails provide a tool where admin user will be able to react on process errors provide a tool where business users in predefined circumstances will be able to perform manual actions with fulfillment process - example: not possible process GTS check as there is no GTS code provided, product content team will need to finalize product configuration Out of scope (boundaries) : it is not role of Fulfillment to set up products and product content in all related systems, this should happen as configuration of those products in respective systems, mapping of products should happen on redesign of fulfillment process for existing products, we are focused on new products on-boarding, this could be part of scope for future, decisions will be made per product Fulfillment hub is not a ESB tool and will not have role of integration platform, PI is still in place and will take care about exposing, controlling and securing all existing and future integration, Fulfillment HUB will heavily use PI interfaces It is not the role of fulfillment hub to gather and consolidate list of active subscriptions / services what customer has It is not role of Fulfillment Hub to trigger upgrade or renewal process, fulfillment of such process can be defined in Fulfillment Hub but trigger mechanism belongs to respective system (eCommerce part or back-end system managing the solution/tenant) Fulfillment Hub is not responsible for order processing (payment capture, invoice creation etc), this are responsibilities of legacy systems, In case there is a problem with payment for the order legacy systems should be able to trigger termination processes in Fulfillment hub Solution Comparision (SWOT) Activiti Engine SAP HANA Cloud Platform Integration SAP Operational Process Inteligence SAP PO SAP Workflow Detailed Analysis'

In [ ]:
# text_sentences1 = clean_text_string_and_break_into_sentences_v1(txt, 256)
# print(len(text_sentences1))
# text_sentences1

In [ ]:
# text_sentences2 = clean_text_string_and_break_into_sentences_v2(txt, 256)
# print(len(text_sentences2))
# text_sentences2

In [ ]:
stop

# EXECUTE FUNCTIONALITY LOOPING OVER THE SOURCE FILES: (v.2)
### LOAD CORP WIKI TEXT, CLEAN DATA, ANNOTATE FILE WITH SAP-ENTITIES, SAVE TO LOCAL FILE

In [ ]:
import numpy as np
np.arange(1,23)

In [ ]:
# -------------------------------------------------------------------------
# 1) LOAD SAP-PRODUCT_ENTITIES:
# -------------------------------------------------------------------------
# PRODUCTS_FOLDER = "sap-products/"
# sap_products_file = "prod_full_sap-L1_v2.csv"

sap_ent_prod =[]
fileObject = open(PRODUCTS_FOLDER+sap_products_file, "r")
sap_ent_prod = fileObject.read()
sap_ent_prod = sap_ent_prod.split('\n')
# sap_ent_prod

# Convert to lower case:
sap_ent_prod_lwr = []
for p in sap_ent_prod:
    p = p.replace('\ufeff','')
    sap_ent_prod_lwr.append(p.lower())

sap_ent_prod_lwr = sorted(set(sap_ent_prod_lwr))

# -------------------------------------------------------------------------
# LOAD CORP WIKI TEXT, CLEAN DATA, ANNOTATE FILE WITH SAP-ENTITIES, SAVE TO LOCAL FILE
# -------------------------------------------------------------------------

# FOLDER = "corp_wiki_raw_texts/"   *** MODED TO TOP OF THE NOTEBOOK
# max_len = 256
# file_number = np.arange(151, 161)

for n in file_number:
    # -------------------------------------------------------------------------
    # LOAD CORP WIKI TEXT
    # -------------------------------------------------------------------------
    print('inicio ', n)
    file_path_from = FOLDER+"corp-wiki-"+str(n)
    print(file_path_from)
    fileObject = open(file_path_from, "r")
    text_string = fileObject.read()
    text_string = text_string.lower()
    #text_string

    # -------------------------------------------------------------------------
    # CLEAN DATA and BREAK INTO SENTENCES OF SIZE
    # -------------------------------------------------------------------------

    text_sentences = clean_text_string_and_break_into_sentences_v2(text_string, max_len)
    # text_sentences

    #release memory
    text_string = []

    # -------------------------------------------------------------------------
    # FIND SAP ENTITIES IN SENTENCES and PRODUCE BILUO ANNOTATED FILE (FINAL)
    # -------------------------------------------------------------------------
    ent_list = []; counter = 1; annotated_sentence = [];
    annotated_text_SAP = []; entity_tag = []; entities_found = []
    ent_found_all = []; words_in_sentence=[];sentence_count=[]; sentences_with_counter=[];
    sentence_count_item=''; sentence_count_full=[]; words_in_sentence_full=[];
    sentence_count_seq=[];words_seq=[]; pos_seq=[];tags_seq=[];
    sentences_list=[];

    from re import search
    
#     from spacy.gold import biluo_tags_from_offsets  # ddeprecated version, replaced by offsets_to_biluo_tags (see below)
    from spacy.training import offsets_to_biluo_tags


    counter = 1

    for sentence in text_sentences:     #full-run

        doc = nlp(sentence)
        #print(doc)
        sentence_count_item = "Sentence: " + str(counter)

        sentence_count = []
        entities_found = []
        words_in_sentence = []
        #loop over prod_entities and verify if they appear on the text
        for ent in sap_ent_prod_lwr:    #full-run
    #     for ent in prod_ent_lwr:          #unit-testing
    
            entity_match_indexes = find_entity_matches(sentence, ent)   # CHANGE-1
            for i in entity_match_indexes:                              # CHANGE-1
                #print('#' + ent +'# found in #'+ sentence)             # CHANGE-1
                #print(ent, '- indexes: ', entity_match_indexes)        # CHANGE-1
                pos_s = i  #position start                              # CHANGE-1
                pos_e = pos_s + len(ent) #position end                  # CHANGE-1
                entity_tag = (pos_s, pos_e, "PROD")                     # CHANGE-1
                entities_found.append(entity_tag)                       # CHANGE-1
                
                # NOTE: CHANGE-1 IS TO BEING ABLE TO FIND MULTIPLE TIME THE SAME SUBSTRING IN A SENTENCE
            
#             if search(ent, sentence):                         # previous-version # CHANGE-1
#                 #print('#' + ent +'# found in #'+ sentence)   # previous-version # CHANGE-1
#                 #print(ent)                                   # previous-version # CHANGE-1
#                 pos_s = sentence.index(ent)  #position start  # previous-version # CHANGE-1
#                 pos_e = pos_s + len(ent) #position end        # previous-version # CHANGE-1
#                 entity_tag = (pos_s, pos_e, "PROD")           # previous-version # CHANGE-1
#                 entities_found.append(entity_tag)             # previous-version # CHANGE-1
#     #             tags = biluo_tags_from_offsets(doc, entity_tag)  # previous-version # CHANGE-1
#     #             entities_found.append(tags)                 # previous-version # CHANGE-1

        #tags  = biluo_tags_from_offsets(doc, entities_found) # deprecated version, replaced by 0ffsets_to_biluo_tags (see below)
        tags = offsets_to_biluo_tags(doc, entities_found)
    #     print(entities_found)
    #     print(tags)

        #ent_found_all.append(tags) # previous-version # CHANGE-2
        #for i in tags:             # previous-version # CHANGE-2
        #    tags_seq.append(i)     # previous-version # CHANGE-2
            
        ent_found_all.append(tags)         # CHANGE-2
        for i in tags:                     # CHANGE-2
            curr_tag = i                   # CHANGE-2
            if curr_tag == 'U-PROD':       # CHANGE-2
                curr_tag = 'B-PROD'        # CHANGE-2
            elif curr_tag == 'L-PROD':     # CHANGE-2
                curr_tag = 'I-PROD'        # CHANGE-2
                
        # NOTE: CHANGE-2 IS CHANGE 'BILOU' TAGS TO 'BIO' TAGGING
        
            tags_seq.append(curr_tag)

        # add each word to another list
        for token in doc:
            sentence_count.append(sentence_count_item)
            sentence_count_seq.append(sentence_count_item)
            words_in_sentence.append(token)
            words_seq.append(token)
            pos_seq.append(token.tag_)
    #     print(sentence_count)
    #     print(words_in_sentence)
    #     sentence_count_full.append(sentence_count)
    #     words_in_sentence_full.append(words_in_sentence)
    #     print('-'*60)   

        counter = counter + 1

    # print(sentence_count_full)   
    # print(words_in_sentence_full)
    # print(ent_found_all)

    # ALL ANNOTATIONS FROM TEXT_STRING (including sentences with Tag = O)
    annotated = {"Sentence": sentence_count_seq, "Word": words_seq, "POS": pos_seq, "Tag": tags_seq}
    annotated_df = pd.DataFrame(annotated)
    # annotated_df

    # Get All Sentences in a list
    sentences_list=[]
    counter = 1
    for sentence in text_sentences: 
        sentence_count_item = "Sentence: " + str(counter)
        counter_sentence = (sentence_count_item, sentence)
        sentences_list.append(counter_sentence)
        counter = counter + 1
    # sentences_list

    # Generate SENTENCES_LIST_DF
    sentences_list_df = pd.DataFrame(sentences_list, columns=['Sentence', 'Text'])
    # sentences_list_df

    # Subset of Annotated sentences with Tag != 'O'
    with_annotations_df = annotated_df[annotated_df['Tag'] == 'B-PROD']
    # with_annotations_df 

    # Counting the number of sentences with true annotations
    #l = len(with_annotations_df.Sentence.unique())
    # print(str(l) + ' unique annotated sentences')

    # Get 'unique annotated sentences' in list "annotated_sentences" (sentences with at least 1 entity found)
    annotated_sentences = []
    annotated_sentences = with_annotations_df.Sentence.unique()
    # annotated_sentences 

    # Get all full sentences with true annotations in df: "annotated_sentences_df"
    annotated_sentences_df =  annotated_df[annotated_df['Sentence'].isin(annotated_sentences)]
    # annotated_sentences_df 

    # Get all sentences from 'sentences_list' that have true annotations
    sentences_list_annot_df  =  sentences_list_df[sentences_list_df['Sentence'].isin(annotated_sentences)]
    # sentences_list_annot_df 
    
    #reset numbering of Sentences in the Annotated Sentences dataframe
    annotated_sentences_df_renumbered = reset_annotated_file_sentence_numbering(annotated_sentences_df, n)
    sentences_list_annot_df_renumbered = reset_annotated_file_sentence_numbering(sentences_list_annot_df, n)
    #annotated_sentences_df_renumbered
    #sentences_list_annot_df_renumbered

    # SAVE TO LOCAL FILE:
    # a) Annotated file:
    file_path_to_a = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences.csv"
    annotated_sentences_df_renumbered.to_csv(file_path_to_a, sep=',',index=False)
    # annotated_sentences_df.to_csv("corp-wiki-1-annotated_sentences.csv", sep=',',index=False)

    # b) Sentences list:
    file_path_to_s = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences_list.csv"
    sentences_list_annot_df_renumbered.to_csv(file_path_to_s, sep=',',index=False)
    # sentences_list_annot_df.to_csv("corp-wiki-1-annotated_sentences_list.csv", sep=',',index=False)
    
    print('fin', n)

In [ ]:
n

In [ ]:
sentence[91:109]

In [ ]:
stop

In [ ]:
len(sentence_count_seq)

In [ ]:
len(words_seq)

In [ ]:
len(pos_seq)

In [ ]:
len(tags_seq)

In [ ]:
sentence[46:53]

In [ ]:
entities_found

In [ ]:
STOP ---- End of code version 2

In [ ]:
frase = str(doc)
# print(frase)
print(frase[2290:2296])
print(frase[2290:2319])

In [ ]:
# sentences_list_annot_df.describe()

In [ ]:
# #reset numbering of Sentences in the Annotated Sentences dataframe
# annotated_sentences_df_renumbered = reset_annotated_file_sentence_numbering(annotated_sentences_df, n)
# sentences_list_annot_df_renumbered = reset_annotated_file_sentence_numbering(sentences_list_annot_df, n)
# #annotated_sentences_df_renumbered
# #sentences_list_annot_df_renumbered

# # SAVE TO LOCAL FILE:
# # a) Annotated file:
# file_path_to_a = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences.csv"
# annotated_sentences_df_renumbered.to_csv(file_path_to_a, sep=',',index=False)
# # annotated_sentences_df.to_csv("corp-wiki-1-annotated_sentences.csv", sep=',',index=False)

# # b) Sentences list:
# file_path_to_s = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences_list.csv"
# sentences_list_annot_df_renumbered.to_csv(file_path_to_s, sep=',',index=False)
# # sentences_list_annot_df.to_csv("corp-wiki-1-annotated_sentences_list.csv", sep=',',index=False)

# print('fin', n)

In [ ]:
## UNIT TESTING CODE:

# strg = 'defect management proces basic principles defects are tracked in each implementation team s jira backlog a defect can be logged by anyone in advanced analytics in any backlog it is the responsibility of the receiving team to investigate dispatch defects to the appropriate developer it is the responsibility of the person logging the defect to provide all the pertinent details to facilitate the defect s correction defect corrections are tracked not by fix versions nor by epics but rather by target release defect reporting will be based on target release teams are free to use epics to group defects if that helps them then when an issue is found by team a related to functional area managed by team b team a logs defect in team b s backlog when possible team a indicates the build number in which the issue was found as well as the target release in which the team believes they need the correction team b defect pipe reviewers are responsible to monitor the pipe and dispatch the defect accordingly to team b developers team b takes the responsibility to communicate back to team a for any delivery dates target release changes defect pipe follow up will be done in bi weekly quality management calls with qa leads defect backlog pipe managers'
# strg = 'defect management proces basic principles defects are tracked in each implementation team'
# print(len(strg))

# nr_wrds = strg.split()
# print(len(nr_wrds))

# s1 = 'defect'
# s2 = 'management'
# s3 = s1 + ' ' + s2
# print(s3)

# max_len = 80
# # if len(nr_wrds) > 5 and len(nr_wrds) > max_len:
# #     print('> 5 and < 128')
# if len(nr_wrds) > max_len:
#     print('strg > max_len')
#     s_trunc1 = strg[:max_len]
#     s_trunc2 = strg[max_len:]
    
# print(s_trunc1)
# print(s_trunc2)


In [ ]:
## UNIT TESTING CODE:
# def smart_truncate(content, length=100, suffix=''):
#     if len(content) <= length:
#         return content
#     else:
#         return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix
    
# truncated1 = smart_truncate(strg, length=50, suffix='')
# print(truncated1)
# print(len(truncated1))
# index_trunc1 = len(truncated1)
# truncated2 = strg[index_trunc1:]
# print(truncated2)

In [ ]:
----------------------------------------------   END OF CODE  ---------------------------------------------------------------

In [ ]:
=============================================================================================================================

# APPENDIX:

## Get CorpWiki article numbers

In [ ]:
import pandas as pd
import numpy as np 
import re

In [ ]:
np.arange(1,2)

In [ ]:
FOLDER = "corp_wiki_raw_texts/"
max_len = 256
file_number = np.arange(1,2)
text_string = ' '

for n in file_number:
    # -------------------------------------------------------------------------
    # LOAD CORP WIKI TEXT
    # -------------------------------------------------------------------------
    print('inicio ', n)
    file_path_from = FOLDER+"corp-wiki-"+str(n)
    print(file_path_from)
    fileObject = open(file_path_from, "r")
    text_string = fileObject.read()
    text_string = text_string.lower()
    text_string = text_string + ". End-corp-wiki-"+str(n)+ '. '
    
print('End')

In [ ]:
s = text_string.lower()
text_string = ''

s = re.sub("@\S+", " ", s)    #remove twitter account
s = re.sub("https*\S+", " ", s)  #remove url
s = s.replace("\t", " ")  # remove tabs
# s = s.replace("\n",". ")  # remove line-breaks

In [ ]:
# s = text_string.lower()
new_s = s.splitlines()
len(new_s)

In [ ]:
test = '2193769388 for development and testing, we have a "test" fs-pm system that\'s setup for us to use for abv and development testing. often times, this is enough, but there are situations where we need to connect to another system (performance testing, load testing). depending on your needs and requirements for the test, contact someone on the fs-pm team to see which system we should use. once a system is identified, we need to do a sanity check to see if this system is usable for pqm. sanity check on new fs-pm system: fs-pm integration user and connection details to the system. availability of business partners integration to commissions (getting help from fs-pm team → contact lars)'

scw_ids = []

for i in new_s:
    scw_ids.append(i[:10])
    
len(scw_ids)

In [ ]:
scw_ids[:30]

In [ ]:
new_s_text_only = []
temp = []

for i in new_s:
    new_line = i.split(i[:10])
    temp.append(new_line)

for t in temp:
    for i in t:
        if i != '':
            i = i.strip() 
            new_s_text_only.append(i)
        
len(new_s_text_only)

In [ ]:
new_s_text_only[:10]

In [ ]:
new_s_text_only_20 = []
for i in new_s_text_only:
    new_s_text_only_20.append(i[:20])
    
new_s_text_only_20

## DEFINE CLEANSING FUNCTIONS (v.1)

In [ ]:
# DEFINE CLEANSING FUNCTIONS:

# -------------------------------------------------------------------------
# function to replace spscific SAP-product strings in text
# -------------------------------------------------------------------------
def replace_strings_in_text(text_string, replacement_list):
    for i in replacement_list:
        new_text = text_string.replace(i[0], i[1])
        text_string = new_text; new_text = ''
    return text_string

# -------------------------------------------------------------------------
# create function to remove numbers in brackets from text_string
# -------------------------------------------------------------------------
def generate_numbers_in_brackets_list(n=200):
    numbers_list = np.arange(1,n)
    brackets_numbers_list = []
    for i in numbers_list:
        string = '[' + str(i) + ']'
        brackets_numbers_list.append(string)
    return brackets_numbers_list

# -------------------------------------------------------------------------
# create function to replace a string with another string
# -------------------------------------------------------------------------
def replace_string_from_texts_list(string, strings_list, replace_by):
    for s in strings_list:
        new_string = string.replace(s, replace_by)
    return new_string

# -------------------------------------------------------------------------
#CLEAN STRING_TEXT (remove special chacters, line-breaks, hashtags, urls, numbers, 
# over spaces,leading/trailing spaces, punctuations
# -------------------------------------------------------------------------
def clean_text_chars(string_text):
    x = string_text
#     y = x.lower()
#     y = re.sub("@\S+", " ", y)    #remove twitter account
#     y = re.sub("https*\S+", " ", y)  #remove url
#     y = y.replace("\t", " ")  # remove tabs
#     y = y.replace("\n",". ")  # remove line-breaks
#     y = re.sub("#\S+", " ", y) # Remove hashtags
#     y = re.sub("\'\w+", '', y) # Remove ticks and the next character
#     #y = re.sub('[%s]' % re.escape(y.punctuation), ' ', y) # Remove punctuations
#     #y = re.sub('\W+',' ', y)  # Remove punctuations
#     y = re.sub('[^A-Za-z0-9.]+', ' ', y)  # Remove all punctuations, except from .
#     #y = re.sub(r'\w*\d+\w*', '', y) # Remove numbers
#     y = re.sub(r'\w*\d+\w*', '', y)
#     y = y.strip()
#     #     y = y.replace('  ', ' ')
#     y = re.sub('\s{2,}', " ", y)  # Replace the over spaces
#     y =  re.sub(r"\b[a-zA-Z]\b", "", y)
#     y = re.sub(r'(?:^| )\w(?:$| )', ' ', y).strip()

    return y

# -------------------------------------------------------------------------
# complete data-cleaning of text (includes: "replace_string_from_texts_list" and "clean_text_chars"functions)
# -------------------------------------------------------------------------
def clean_text_complete(input_text):
    #generate list of numbers in brackets (this normally appear in most wiki)
    brackets_numbers_list = generate_numbers_in_brackets_list(n=200)
    
    #remove those numbers in brackes from the text
    text_string2 = replace_string_from_texts_list(text_string, brackets_numbers_list,'')
    
    #clean text chars
    cleaned_text_string = clean_text_chars(text_string2)
    
    return cleaned_text_string

# -------------------------------------------------------------------------
# SPLIT DATA_STRING INTO SENTENCES
# -------------------------------------------------------------------------
def split_text_string_into_sentences(text_string):
    text_sentences = []
    text_sentences_tmp = text_string.split('.')
    for s in text_sentences_tmp:
        if len(s) > 5:
            s_nlts = s.strip()
            s_nlts = s_nlts.replace('     ',' ') # replace 5 spaces by 1 space
            s_nlts = s_nlts.replace('    ',' ') # replace 4 spaces by 1 space
            s_nlts = s_nlts.replace('   ',' ') # replace 3 spaces by 1 space
            s_nlts = s_nlts.replace('  ',' ') # replace 2 spaces by 1 space
            text_sentences.append(s_nlts)

    return text_sentences


# -------------------------------------------------------------------------
# RESET SENTENCE NUMBERING OF ANNOTATED SENTENCES DATAFRAME
# -------------------------------------------------------------------------
def reset_annotated_file_sentence_numbering(df_data, n):
    df_data = df_data.reset_index(drop=True)
    previous_sentence = ''
    counter = 1
    sentence_counter = 'Sentence: '+str(n)+'_'+str(counter)

    for i in df_data.index: 
        if i == 0:
            previous_sentence = df_data['Sentence'][i]
            df_data['Sentence'][i] = 'Sentence: '+str(n)+'_1'

        else:
            if df_data['Sentence'][i] == previous_sentence:
                previous_sentence = df_data['Sentence'][i]
                df_data['Sentence'][i] = sentence_counter
            else:
                counter = counter + 1
                sentence_counter = 'Sentence: '+str(n)+'_'+str(counter)
                previous_sentence = df_data['Sentence'][i]
                df_data['Sentence'][i] = sentence_counter

    return df_data

# EXECUTE FUNCTIONALITY LOOPING OVER THE SOURCE FILES: (v.1)
### LOAD CORP WIKI TEXT, CLEAN DATA, ANNOTATE FILE WITH SAP-ENTITIES, SAVE TO LOCAL FILE

In [ ]:
# -------------------------------------------------------------------------
# LOAD CORP WIKI TEXT
# -------------------------------------------------------------------------
file_number = np.arange(101,111)
for n in file_number:
    print('inicio ', n)
    file_path_from = "corp_wiki_texts/corp-wiki-"+str(n)
    print(file_path_from)
    fileObject = open(file_path_from, "r")
    text_string = fileObject.read()
    text_string = text_string.lower()
    #text_string

    # -------------------------------------------------------------------------
    # CLEAN DATA
    # -------------------------------------------------------------------------
    cleaned_text_string = clean_text_complete(text_string)
    # cleaned_text_string

    # -------------------------------------------------------------------------
    # EXECUTE SAP-PRODUCTS RELATED STRING REPLACEMENTS:
    # -------------------------------------------------------------------------
    cleaned_text_string = replace_strings_in_text(cleaned_text_string,replace)
    # cleaned_text_string

    #release memory
    text_string = []

    # -------------------------------------------------------------------------
    # SPLIT DATA_STRING INTO SENTENCES
    # -------------------------------------------------------------------------
    text_sentences = split_text_string_into_sentences(cleaned_text_string)
    # text_sentences

    #release memory
    cleaned_text_string = []

    # -------------------------------------------------------------------------
    # FIND SAP ENTITIES IN SENTENCES and PRODUCE BILUO ANNOTATED FILE (FINAL)
    # -------------------------------------------------------------------------
    ent_list = []; counter = 1; annotated_sentence = [];
    annotated_text_SAP = []; entity_tag = []; entities_found = []
    ent_found_all = []; words_in_sentence=[];sentence_count=[]; sentences_with_counter=[];
    sentence_count_item=''; sentence_count_full=[]; words_in_sentence_full=[];
    sentence_count_seq=[];words_seq=[]; pos_seq=[];tags_seq=[];
    sentences_list=[];

    from re import search
    #from spacy.gold import biluo_tags_from_offsets# deprecated version, replaced by 0ffsets_to_biluo_tags (see below)
    from spacy.training import offsets_to_biluo_tags

    counter = 1

    for sentence in text_sentences:     #full-run
    # for sentence in text_sentences_lwr:   #unit-testing
        sentence = sentence.replace('sap_hanalite', '')
        sentence = sentence.replace('sap hana_xs', 'sap hana_xsa')
        doc = nlp(sentence)
        print(doc)
        sentence_count_item = "Sentence: " + str(counter)

        sentence_count = []
        entities_found = []
        words_in_sentence = []
        #loop over prod_entities and verify if they appear on the text
        for ent in sap_ent_prod_lwr:    #full-run
    #     for ent in prod_ent_lwr:          #unit-testing
            if search(ent, sentence):
                #print('#' + ent +'# found in #'+ sentence)
                #print(ent)
                pos_s = sentence.index(ent)  #position start
                pos_e = pos_s + len(ent) #position end
                entity_tag = (pos_s, pos_e, "PROD")
                entities_found.append(entity_tag)
    #             tags = biluo_tags_from_offsets(doc, entity_tag)
    #             entities_found.append(tags)

        #tags  = biluo_tags_from_offsets(doc, entities_found)  # deprecated version, replaced by 0ffsets_to_biluo_tags (see below)
        tags = offsets_to_biluo_tags(doc, entities_found)
    #     print(entities_found)
    #     print(tags)

        ent_found_all.append(tags)
        for i in tags:
            tags_seq.append(i)

        # add each word to another list
        for token in doc:
            sentence_count.append(sentence_count_item)
            sentence_count_seq.append(sentence_count_item)
            words_in_sentence.append(token)
            words_seq.append(token)
            pos_seq.append(token.tag_)
    #     print(sentence_count)
    #     print(words_in_sentence)
    #     sentence_count_full.append(sentence_count)
    #     words_in_sentence_full.append(words_in_sentence)
    #     print('-'*60)   

        counter = counter + 1

    # print(sentence_count_full)   
    # print(words_in_sentence_full)
    # print(ent_found_all)

    # ALL ANNOTATIONS FROM TEXT_STRING (including sentences with Tag = O)
    annotated = {"Sentence": sentence_count_seq, "Word": words_seq, "POS": pos_seq, "Tag": tags_seq}
    annotated_df = pd.DataFrame(annotated)
    # annotated_df

    # Get All Sentences in a list
    sentences_list=[]
    counter = 1
    for sentence in text_sentences: 
        sentence_count_item = "Sentence: " + str(counter)
        counter_sentence = (sentence_count_item, sentence)
        sentences_list.append(counter_sentence)
        counter = counter + 1
    # sentences_list

    # Generate SENTENCES_LIST_DF
    sentences_list_df = pd.DataFrame(sentences_list, columns=['Sentence', 'Text'])
    # sentences_list_df

    # Subset of Annotated sentences with Tag != 'O'
    with_annotations_df = annotated_df[annotated_df['Tag'] != 'O']
    # with_annotations_df 

    # Counting the number of sentences with true annotations
    l = len(with_annotations_df.Sentence.unique())
    # print(str(l) + ' unique annotated sentences')

    # Get 'unique annotated sentences' in list "annotated_sentences" (sentences with at least 1 entity found)
    annotated_sentences = []
    annotated_sentences = with_annotations_df.Sentence.unique()
    # annotated_sentences 

    # Get all full sentences with true annotations in df: "annotated_sentences_df"
    annotated_sentences_df =  annotated_df[annotated_df['Sentence'].isin(annotated_sentences)]
    # annotated_sentences_df 

    # Get all sentences from 'sentences_list' that have true annotations
    sentences_list_annot_df  =  sentences_list_df[sentences_list_df['Sentence'].isin(annotated_sentences)]
    # sentences_list_annot_df 
    
    #reset numbering of Sentences in the Annotated Sentences dataframe
    annotated_sentences_df_renumbered = reset_annotated_file_sentence_numbering(annotated_sentences_df, n)
    sentences_list_annot_df_renumbered = reset_annotated_file_sentence_numbering(sentences_list_annot_df, n)
    #annotated_sentences_df_renumbered
    #sentences_list_annot_df_renumbered

    # SAVE TO LOCAL FILE:
    # a) Annotated file:
    file_path_to_a = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences.csv"
    annotated_sentences_df_renumbered.to_csv(file_path_to_a, sep=',',index=False)
    # annotated_sentences_df.to_csv("corp-wiki-1-annotated_sentences.csv", sep=',',index=False)

    # b) Sentences list:
    file_path_to_s = "corp_wiki_annotated/corp-wiki-"+str(n)+"-annotated_sentences_list.csv"
    sentences_list_annot_df_renumbered.to_csv(file_path_to_s, sep=',',index=False)
    # sentences_list_annot_df.to_csv("corp-wiki-1-annotated_sentences_list.csv", sep=',',index=False)
    
    print('fin', n)

# NAMED ENTITY RECOGNITION with SPACY:

### Find the pre-trained entities

#### Sample text:

In [ ]:
#Processing the Named Entity Reconition with SpaCy
# `doc.ents()` function returns the entities from a given text stored in doc variable
# `doc.ents.text` and `doc.ents.label_` return the name of the entities and label or entity type respectively

#SAMPLE: 

#Calling the nlp object on a string of text will return a processed Doc:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
print('sample sentence:  ', doc)

print('-'*50)
print('NER (Named Entity Recognition)')
print('-'*50)
# Get Entities Information
print([(X.text, X.label_) for X in doc.ents]) 

# using the Spacy Named Entity Recognition visualizer
#spacy.displacy.serve(doc, style='ent')

#POS (Part-of-Speech tags)
print('-'*50)
print('#POS (Part-of-Speech tags)')
print('-'*50)
for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)
    print(token.text, token.tag_)

## Apply to given text and produce annotated file with pre-trained Standard Entities

In [ ]:
# ent_list = []
# counter = 1
# annotated_sentence = []
# annotated_text = []

# for s in text_sentences:
#     doc = nlp(s)
#     ent_list = [(X.text, X.label_) for X in doc.ents]
#     #example: ('Thousands', 'CARDINAL'), ('London', 'GPE'), ('Iraq', 'GPE'), ('British', 'NORP')]
#     for token in doc:
#         tag = ''
#         annotated_sentence.append('Sentence: '+str(counter))   #column: Sentence #
# #         annotated_sentence.append(s)
#         annotated_sentence.append(token.text)                  #column: Word           
#         annotated_sentence.append(token.tag_)                  # column: POS
#         word = str(token)
# #         print('-'*10)
# #         print(word)
#         for i in ent_list:
# #             print(token)
# #             print(i[0])
# #             print(i[1])
# #         print('-'*10)
            
#             ent_w = str(i[0])
#             ent_t = str(i[1])
# #             print(entity)
#             if word == ent_w :
# #                 print('word equals entity')
#                 tag = ent_t

#         if tag == '':
#             tag = 'O'
        
#         annotated_sentence.append(tag)                          #column: Tag

#     annotated_text.append(annotated_sentence)
    
# annotated_text


### SAVE Standard Entities Annotated list to Local File 

In [ ]:
# import pickle

# with open("corp_wiki_texts/corp-wiki-1-annotated.txt", "wb") as fp:   #Pickling
#     pickle.dump(annotated_text, fp)

In [ ]:
# #release memory
# annotated_text = []

### LOAD Standard Entities Annotated list to Local File 

In [ ]:
# import pickle
# with open("corp_wiki_texts/corp-wiki-1-annotated.txt", "rb") as data:
#     W = pickle.load(data)

In [ ]:
# W

In [ ]:
#clear table
#W = []

In [ ]:
# import pandas as pd
# import numpy as np

In [ ]:
# df_data = pd.DataFrame(W, columns=['Sentence #','Word','POS','Tag'])

In [ ]:
# df_data.describe()

## Find and Annotate SAP Entities

### Small Example for building the code

In [ ]:
# # SMALL DATA SAMPLES FOR UNIT TESTING:

# prod_entities = ['SAP Cloud Foundry','SAP Conversational AI','SAP S/4HANA','SAP Cloud Platform','SAP Gateway','SAP HANA']
# text_sentences = ['SAP S/4HANA runs on SAP HANA database and can be extended on SAP Cloud Foundry','SAP Conversational AI can be integrated with SAP S/4HANA via SAP Cloud Foundry']



In [ ]:
# # Convert to lower case:
# prod_ent_sample_lwr = []
# for p in prod_entities_sample:
#     prod_ent_sample_lwr.append(p.lower())
    
# print(prod_ent_sample_lwr)

# text_sentences_lwr = []
# for s in text_sentences_sample:
#     text_sentences_sample_lwr.append(s.lower())
    
# print(text_sentences_sample_lwr)

### UNIT TEST: FIND SAP ENTITIES IN SENTENCES and PRODUCE BILUO TAGS

In [ ]:
# # FIND SAP ENTITIES IN SENTENCES and PRODUCE BILUO TAGS
# ent_list = []; counter = 1; annotated_sentence = [];
# annotated_text_SAP = []; entity_tag = []; entities_found = []
# ent_found_all = []

# from re import search
# from spacy.gold import biluo_tags_from_offsets

# for sentence in text_sentences_lwr:
#     doc = nlp(sentence)
#     print(doc)
#     entities_found = []
#     #loop over prod_entities and verify if they appear on the text
#     for ent in prod_ent_lwr:
#         if search(ent, sentence):
#             #print('#' + ent +'# found in #'+ sentence)
#             print(ent)
#             pos_s = sentence.index(ent)  #position start
#             pos_e = pos_s + len(ent) #position end
#             entity_tag = (pos_s, pos_e, "PROD")
#             entities_found.append(entity_tag)
# #             tags = biluo_tags_from_offsets(doc, entity_tag)  # deprecated version, replaced by 0ffsets_to_biluo_tags (see below)
              #tags = offsets_to_biluo_tags(doc, entities_found)
# #             entities_found.append(tags)
    
#     tags  = biluo_tags_from_offsets(doc, entities_found)
#     print(entities_found)
#     print(tags)
#     print('-'*60)
#     ent_found_all.append(entities_found)
    
# print(ent_found_all)

In [ ]:
## OUTPUT FROM ABOVE CELL:

# sap s/4hana runs on sap hana database and can be extended on sap cloud foundry
# sap cloud foundry
# sap s/4hana
# sap hana
# [(61, 78, 'PROD'), (0, 11, 'PROD'), (20, 28, 'PROD')]
# ['B-PROD', 'L-PROD', 'O', 'O', 'B-PROD', 'L-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'L-PROD']
# ------------------------------------------------------------
# sap conversational ai can be integrated with sap s/4hana via sap cloud foundry
# sap cloud foundry
# sap conversational ai
# sap s/4hana
# [(61, 78, 'PROD'), (0, 21, 'PROD'), (45, 56, 'PROD')]
# ['B-PROD', 'I-PROD', 'L-PROD', 'O', 'O', 'O', 'O', 'B-PROD', 'L-PROD', 'O', 'B-PROD', 'I-PROD', 'L-PROD']
# ------------------------------------------------------------
# [[(61, 78, 'PROD'), (0, 11, 'PROD'), (20, 28, 'PROD')], [(61, 78, 'PROD'), (0, 21, 'PROD'), (45, 56, 'PROD')]]

